In [ ]:
!pip install attrdict
!pip install -q -U transformers
!pip install soynlp
!pip install emoji
# !pip uninstall torch
# !pip uninstall torch # run this command twice
# !pip install torch===1.10.0
# !pip install git+https://github.com/kakaobrain/pororo.git   # kakao pororo 라이브러리 사용

In [ ]:
# hanspell 라이브러리 사용
!pip install git+https://github.com/ssut/py-hanspell.git
# !python py-hanspell/setup.py install    # setup.py line7 수정 /content/py-hanspell/requirements.txt 

In [ ]:
# from pororo import Pororo # 카카오 포로로
import pandas as pd
import os
import json
import numpy as np
import shutil
from hanspell import spell_checker  # 한스펠

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from datetime import datetime, timezone, timedelta
import random
from tqdm import tqdm

from attrdict import AttrDict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils import *
from torch.utils.data import DataLoader
from torch.optim import Adam, AdamW

import re
import emoji
from soynlp.normalizer import repeat_normalize

from transformers import logging, get_linear_schedule_with_warmup

from transformers import ( 
    BertConfig,
    ElectraConfig
)

from transformers import (
    BertTokenizer,  
    AutoTokenizer,
    ElectraTokenizer,
    AlbertTokenizer
)

from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification
)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Seed 고정
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
seed_everything(2022)

In [ ]:
# 사용할 GPU 지정
print("number of GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
print("Does GPU exist? : ", use_cuda)
DEVICE = torch.device("cuda" if use_cuda else "cpu")

number of GPUs:  1
Does GPU exist? :  True


In [ ]:
# True 일 때 코드를 실행하면 example 등을 보여줌
DEBUG = True

# 경로
DATA_PATH = '/content/drive/MyDrive/AI_team/competition3'

## BIAS Prediction 진행

In [ ]:
# config 파일 불러오기 / bias
config_path_bias = os.path.join(DATA_PATH + '/baseline2/config_bias_stratified.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = AttrDict(json.load(f))
            print("config file loaded.")
            print(args.pretrained_model)
    else:
        assert False, 'config json file cannot be found.. please check the path again.'
    return args
    
# 코드 중간중간에 끼워넣어 리셋 가능
args_bias = set_config(config_path_bias)

# 결과 저장 폴더 미리 생성
os.makedirs(args_bias.result_dir, exist_ok=True)
os.makedirs(args_bias.config_dir, exist_ok=True)

config file loaded.
beomi/beep-KcELECTRA-base-bias


#### train 과 test csv파일 읽어오기

In [ ]:
# Train 파일
train_path = os.path.join(args_bias.data_dir,'train.csv')
print("train 데이터 경로가 올바른가요? : ", os.path.lexists(train_path))
train_df = pd.read_csv(train_path, encoding = 'UTF-8-SIG')

# Test 파일
test_path = os.path.join(args_bias.data_dir,'test.csv')
print("test 데이터 경로가 올바른가요? : ", os.path.lexists(test_path))
test_df = pd.read_csv(test_path)

train 데이터 경로가 올바른가요? :  True
test 데이터 경로가 올바른가요? :  True


#### BIAS 데이터 전처리(Label Encoding)

In [ ]:
bias_combinations = np.array(train_df.bias.unique()).T.reshape(-1,1)
bias_labeling = list(np.array([train_df['bias'].values]).T.reshape(-1,1))

bias_labels = []
for i, arr in enumerate(bias_labeling):
    for idx, elem in enumerate(bias_combinations):
        if np.array_equal(elem, arr):
            bias_labels.append(idx)

train_df['label'] = bias_labels

In [ ]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
punct = "/-'.,#$%\'()*+-!?/:·;→<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' + f'{emojis}' # !, ? 넣을지 말지 고민중
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", 'ᆢ':'', "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 
            '넘도':'놈도', 'OOO' : 'ㅅㅂ', 'ㄹㅇ':'정말', '❤️':'♥', '♡':'♥', '싹수':' 싸가지', '와꾸':'외모',
            

            
            }    # '♥': ' 하트 ', '★': ' 스타 '
def clean_punc(text, punct, mapping): 
    for p in mapping: 
        text = text.replace(p, mapping[p]) 
    for p in punct:
        text = text.replace(p, ' ')
    for s in specials:
        text = text.replace(s, specials[s])
    normalized_text = repeat_normalize(text, num_repeats=2)
    if len(str(normalized_text)) < 9:
        text = (normalized_text + ' ') * 3
    elif 6 <= len(str(normalized_text)) < 13:
        text = (normalized_text + ' ') * 2
    else:
        text = normalized_text
    return text

In [ ]:
train_df['clean_comment'] = train_df['comment'].apply(lambda x: clean_punc(x, punct, punct_mapping))

In [ ]:
# 15글자 미만 확인
train_df[train_df['clean_comment'].str.len() < 15 ]

,title,comment,bias,hate,label,clean_comment
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0,김태리 정말 연기잘해 진짜
6,"""[POP이슈]'프듀2' 김사무엘 父, 멕시코서 숨진 채 발견→타살 의혹 제기→애도...",연예계에 외국인노동자 많네..,others,hate,1,연예계에 외국인노동자 많네
18,[종합IS] 서유리♥최병길 결혼 발표…최초 공개 쏟아진 '비스',결혼 축하합니다~ 행복하세요,none,none,0,결혼 축하합니다 행복하세요
19,"""[SC무비] 이승기X심은경 `궁합` 100만 돌파 """"로맨스 잔혹사 끊었다""""""",월트디즈니 만화를 보는듯...,none,none,0,월트디즈니 만화를 보는듯
22,"""[룩@차이나] 中관계자 """"판빙빙, 조사받고 귀가한 지 한참 됐다""""""",무슨상관이냐고 기자양반ㅠ,none,none,0,무슨상관이냐고 기자양반ㅠ
...,...,...,...,...,...,...
8224,"""[엑's PICK] '인생술집' 윤소희 """"과학고 조기졸업, 김태희 때문에 카이스...",학사비리 있을기다 추적해바,others,hate,1,학사비리 있을기다 추적해바
8340,"""[단독] 인피니트 장동우, 의경 불합격…현역으로 입대""",꼬시다ㅋㅋㅋㅋㅋㅋ근데 얜 누구징;,none,hate,0,꼬시다ㅋㅋ근데 얜 누구징
8342,"""""""톱모델의 위엄""""…한혜진, 비현실적 비키니 몸매""",다음 세상엔 저 몸으로 태.........................,none,none,0,다음 세상엔 저 몸으로 태
8346,"""씨잼, 대마초 흡연 사실 시인... 동료 래퍼 · 프로듀서도 검거""",비와이는 예수님이 지켰다,none,none,0,비와이는 예수님이 지켰다


### 카카오 Pororo 전처리 ( 전혀 도움안됨 )

In [ ]:
# 한글 맞춤법(띄어쓰기) 수정
spacing = Pororo(task="gec", lang="ko")

# 오류 문장 입력
changed_sentence = spacing("아버지가방에들어간다.")

As of now, this beta model tries to correct spacing errors in Korean text.


In [ ]:
changed_sentence

'아버지가 방에 들어간다.'

In [ ]:
train_df['clean_comment2'] = train_df['clean_comment'].apply(lambda x: spacing(x))

In [ ]:
train_df

,title,comment,bias,hate,label,clean_comment,clean_comment2
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0,김태리 정말 연기잘해 진짜,김태리 정말 연기 잘해 진짜
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,0,공효진 발연기나이질생각이읍던데 왜계속주연일까,공효진 발연기나이질 생각이읍던데 왜 계속 주연일까
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,1,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감...,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니순이 아니니깐 작은 거...
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0,일본 축구 져라,일본 축구 져라
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0,난 절대로 임현주 욕하는인간이랑은 안논다,난 절대로 임현주 욕하는 인간이랑은 안 논다
...,...,...,...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,1,아니 근데 튜닝한사람은 프러포즈받지도 결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,아니근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함?ㅋㅋㅋ지들은 돈 없어서 못하는 ...
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,2,그러니깐 여자를 잘만나야되 징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯 근데 ...,그러니깐 여자를 잘 만나야되 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯근...
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,0,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다♡♡♡
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none,0,재미가 없어요,재미가 없어요


### 한스펠(hanspell) 전처리

In [ ]:
result = spell_checker.check(train_df['clean_comment'][2])
result

Checked(result=True, original='누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감사하고 항상 좋은일만 가득하길', checked='누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 거에 감사하고 항상 좋은 일만 가득하길', errors=5, words=OrderedDict([('누구처럼', 0), ('돈만', 0), ('밝히는', 0), ('저급', 2), ('인생은', 2), ('살아가지', 2), ('마시길', 2), ('행복은', 0), ('머니', 2), ('순이', 2), ('아니니깐', 0), ('작은', 2), ('거에', 2), ('감사하고', 0), ('항상', 0), ('좋은', 2), ('일만', 2), ('가득하길', 0)]), time=0.8556177616119385)

In [ ]:
result = spell_checker.check(train_df['clean_comment'][24])
result.as_dict()['checked']

'살 빼니까 왜 이렇게 느끼하지ㅋㅋ'

In [ ]:
result = spell_checker.check(train_df['clean_comment'][7484])
result.as_dict()['checked']

'양지원이랑 조정석은 옛날에 사귀었었어요 뮤지컬 하면서 거미 만나기 전 조정석 유명하기 전에'

In [ ]:
train_df

In [ ]:
# train 전처리
train_df['clean_comment2'] = train_df['clean_comment'].apply(lambda x: spell_checker.check(x).as_dict()['checked'])

In [ ]:
# 한스펠 전처리한 csv 파일 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data2/hanspell_comment.csv')

In [ ]:
train_df

,title,comment,bias,hate,label,clean_comment,clean_comment2
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0,김태리 정말 연기잘해 진짜,김태리 정말 연기 잘해 진짜
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,0,공효진 발연기나이질생각이읍던데 왜계속주연일까,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,1,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감...,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0,일본 축구 져라 일본 축구 져라 일본 축구 져라,일본 축구 져라 일본 축구 져라 일본 축구 져라
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0,난 절대로 임현주 욕하는인간이랑은 안논다,난 절대로 임현주 욕하는 인간이랑은 안 논다
...,...,...,...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,1,아니 근데 튜닝한사람은 프러포즈받지도 결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함? ᄏᄏᄏ 자기들은 돈 없어서 ...
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,2,그러니깐 여자를 잘만나야되 징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯 근데 ...,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,0,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none,0,재미가 없어요 재미가 없어요 재미가 없어요,재미가 없어요 재미가 없어요 재미가 없어요


In [ ]:
# test 전처리
test_df['clean_comment'] = test_df['comment'].apply(lambda x: clean_punc(x, punct, punct_mapping))

In [ ]:
test_df

,ID,title,comment,clean_comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,누군데 얘네? 누군데 얘네? 누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 짜식아!음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...
...,...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...


In [ ]:
# test_df 한스펠 전처리
test_df['clean_comment2'] = test_df['clean_comment'].apply(lambda x: spell_checker.check(x).as_dict()['checked'])

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data2/hanspell_comment_test.csv')

In [ ]:
test_df

,ID,title,comment,clean_comment,clean_comment2
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요,둘 다 너무 좋다 행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만 원 이하는 현금결제만 하라고 써놓은 집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,누군데 얘네? 누군데 얘네? 누군데 얘네?,누군데 얘네? 누군데 얘네? 누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 짜식아!음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,쇼 하지 마라 자식아! 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 ...
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은 건가요? 수술하면 이렇게 자연스러워진다...
...,...,...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야 할걸?
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야지 콘셉트가 바뀌니깐 재미 지네
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,성형으로 다 뜯어고쳐놓고 예쁜 척 성형 전 네 얼굴 다 알고 있다 순자처럼 된장 냄...
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀 다른 전개던데 무슨ㅋㅋㄱ 우리나라 사람들은 분위기만 비슷하...


#### BIAS Dataset 준비

In [ ]:
BIAS_TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer,
    "AutoTokenizer": AutoTokenizer,
    "ElectraTokenizer": ElectraTokenizer,
    "AlbertTokenizer": AlbertTokenizer
}

BIAS_TOKENIZER = BIAS_TOKENIZER_CLASSES[args_bias.tokenizer_class].from_pretrained(args_bias.pretrained_model)
if DEBUG==True:
    print(BIAS_TOKENIZER_CLASSES)

Downloading:   0%|          | 0.00/544 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


{'BertTokenizer': <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>, 'AutoTokenizer': <class 'transformers.models.auto.tokenization_auto.AutoTokenizer'>, 'ElectraTokenizer': <class 'transformers.models.electra.tokenization_electra.ElectraTokenizer'>, 'AlbertTokenizer': <class 'transformers.utils.dummy_sentencepiece_objects.AlbertTokenizer'>}


### test 데이터 토큰 확인

In [ ]:
example = test_df['clean_comment2'][0]
print(BIAS_TOKENIZER(example))

{'input_ids': [2, 983, 820, 8072, 10367, 9554, 8479, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
print(BIAS_TOKENIZER.encode(example),"\n")
    
# 토큰으로 나누기
print(BIAS_TOKENIZER.tokenize(example),"\n")
    
# 토큰 id로 매핑하기
print(BIAS_TOKENIZER.convert_tokens_to_ids(BIAS_TOKENIZER.tokenize(example)))

[2, 983, 820, 8072, 10367, 9554, 8479, 3] 

['둘', '다', '너무', '좋다', '행복', '##하세요'] 

[983, 820, 8072, 10367, 9554, 8479]


In [ ]:
example2 = '[UNK]'
# 토큰으로 나누기
print(BIAS_TOKENIZER.tokenize(example2),"\n")
    
# 토큰 id로 매핑하기
print(BIAS_TOKENIZER.convert_tokens_to_ids(BIAS_TOKENIZER.tokenize(example2)))

['[UNK]'] 

[1]


In [ ]:
test_df['token'] = test_df['clean_comment2'].apply(lambda x: BIAS_TOKENIZER.tokenize( x ))
test_df['token_id'] = test_df['clean_comment2'].apply(lambda x: BIAS_TOKENIZER.convert_tokens_to_ids(BIAS_TOKENIZER.tokenize( x )))
test_df2 = test_df.drop(['title', 'clean_comment'], axis=1)
test_df2.drop('comment', axis=1, inplace=True)

In [ ]:
test_df2.iloc[:, :3]

,ID,clean_comment2,token
0,0,둘 다 너무 좋다 행복하세요,"[둘, 다, 너무, 좋다, 행복, ##하세요]"
1,1,근데 만 원 이하는 현금결제만 하라고 써놓은 집 우리나라에 엄청 많은데,"[근데, 만, 원, 이하, ##는, 현금, ##결제, ##만, 하라고, 써, ##놓..."
2,2,누군데 얘네? 누군데 얘네? 누군데 얘네?,"[누군데, 얘네, ?, 누군데, 얘네, ?, 누군데, 얘네, ?]"
3,3,쇼 하지 마라 자식아! 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 ...,"[쇼, 하지, 마라, 자식, ##아, !, 음주, 1번, ##은, 실수, 2번, #..."
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은 건가요? 수술하면 이렇게 자연스러워진다...,"[안, ##검, ##하, ##수, 가지고, 있는, 분, ##께, 희망을, 주고, 싶..."
...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야 할걸?,"[얜, 그냥, 봐도, 아니다, ㅋ, 고소, ##당하면, 어마어마한, 금액, 물어, ..."
507,507,대박 게스트 꼭 봐야지 콘셉트가 바뀌니깐 재미 지네,"[대박, 게스트, 꼭, 봐야지, 콘셉트, ##가, 바뀌, ##니깐, 재미, 지네]"
508,508,성형으로 다 뜯어고쳐놓고 예쁜 척 성형 전 네 얼굴 다 알고 있다 순자처럼 된장 냄...,"[성형, ##으로, 다, 뜯어, ##고, ##쳐, ##놓고, 예쁜, 척, 성형, 전..."
509,509,분위기는 비슷하다만 전혀 다른 전개던데 무슨ㅋㅋㄱ 우리나라 사람들은 분위기만 비슷하...,"[분위기, ##는, 비슷, ##하다, ##만, 전혀, 다른, 전개, ##던데, 무슨..."


In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len, mode = 'train'):
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode   
        if self.mode !='test':
            try: 
                self.labels = df['label'].tolist()
            except:
                assert False, 'CustomDataset Error : \'label\' column does not exist in the dataframe'
     
    def __len__(self):
        return len(self.data)
                
    def __getitem__(self, idx):
        """
        전체 데이터에서 특정 인덱스 (idx)에 해당하는 기사제목과 댓글 내용을 
        토크나이즈한 data('input_ids', 'attention_mask','token_type_ids')의 딕셔너리 형태로 불러옴
        """
        # title = self.data.clean_title.iloc[idx] #clean_title
        comment = self.data.clean_comment2.iloc[idx]    # clean_comment2
        
        tokenized_text = self.tokenizer(comment, # title 제거 self.tokenizer(title, comment,
                             padding= 'max_length',
                             max_length=self.max_len,
                             truncation=True,
                             return_token_type_ids=False,   # 원본 True
                             return_attention_mask=True,
                             return_tensors = "pt")
        
        data = {'input_ids': tokenized_text['input_ids'].clone().detach().long(),
               'attention_mask': tokenized_text['attention_mask'].clone().detach().long(),
               'token_type_ids': tokenized_text['token_type_ids'].clone().detach().long(),}
        
        if self.mode != 'test':
            label = self.data.label.iloc[idx]
            return data, label
        else:
            return data

#### 데이터 스플릿

In [ ]:
# dataset (bias, hate 공통적으로 사용) -> stratified 로 나눠줄 예정
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args_bias.seed)

train_dataset = CustomDataset(train_data, BIAS_TOKENIZER, args_bias.max_seq_len, 'train')
val_dataset = CustomDataset(val_data, BIAS_TOKENIZER, args_bias.max_seq_len, 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

Train dataset:  7530
Validation dataset:  837


In [ ]:
def split_stratified_shuffle_split(df, fold, n_split, input_col='comment', target_col='hate', seed=2022):
    skf = StratifiedShuffleSplit(n_splits=n_split, test_size=1/n_split, random_state=seed)
    for idx, (train_index, valid_index) in enumerate(skf.split(df[input_col], df[target_col])):
        # print(train_index)
        # print(valid_index)
        # print(' ')
        if idx == fold:
            _train, _valid = train_index, valid_index
    return _train, _valid, skf


In [ ]:
train_index, valid_index, skf = split_stratified_shuffle_split(df=train_df, fold=0, n_split=5)

[4702 3372 1663 ... 7609 1606 4525]
[7718 1413 5726 ...   17 3072 2934]
 
[6276 4159 7325 ... 5482 1018 7936]
[3243 7728 4349 ... 6099 7950 6762]
 
[6564  317  772 ... 1196 6505 3389]
[2122 3785 6368 ...  103 4843 7344]
 
[7338 3795 7827 ... 8312 7044 5196]
[1336 1462 3333 ... 3443 8352 5111]
 
[ 883 5678  196 ... 2512 2021 6228]
[5346 4993 3152 ... 6545 3056  698]
 


In [ ]:
train_data, val_data = train_df.iloc[train_index], train_df.iloc[valid_index]
train_data.shape[0]

#### 모델 불러오기

In [ ]:
from transformers import logging
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification,
    "AutoModel": AutoModel,
    "ElectraForSequenceClassification": ElectraForSequenceClassification,
    "AlbertForSequenceClassification": AlbertForSequenceClassification,

}


bias_myModel = BASE_MODELS[args_bias.architecture].from_pretrained(args_bias.pretrained_model, 
                                                         num_labels = args_bias.num_classes, 
                                                         output_attentions = False, # Whether the model returns attentions weights.
                                                         output_hidden_states = True # Whether the model returns all hidden-states.
                                                        )
if DEBUG==True:
    # 모델 구조 확인
    print(bias_myModel)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
class myClassifier(nn.Module):
    def __init__(self, model, hidden_size = 768, num_classes=3, selected_layers=False, params=None, args=None):
        super(myClassifier, self).__init__()
        self.model = model
        self.softmax = nn.Softmax(dim=1) 
        self.selected_layers = selected_layers
        self.args = args
        
        # 사실 dr rate은 model config 에서 hidden_dropout_prob로 가져와야 하는데 bert에선 0.1이 쓰였음
        self.dropout = nn.Dropout(0.1)


    def forward(self, token_ids, attention_mask, segment_ids):      
        outputs = self.model(input_ids = token_ids, 
                             token_type_ids = segment_ids.long(), 
                             attention_mask = attention_mask.float().to(token_ids.device))
        
        # hidden state에서 마지막 4개 레이어를 뽑아 합쳐 새로운 pooled output 을 만드는 시도
        if self.selected_layers == True:
            hidden_states = outputs.hidden_states
            pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
            # print("concatenated output shape: ", pooled_output.shape)
            ## dim(batch_size, max_seq_len, hidden_dim) 에서 가운데를 0이라 지정함으로, [cls] 토큰의 임베딩을 가져온다. (text classification 구조 참고)
            pooled_output = pooled_output[:, 0, :]
            # print(pooled_output)

            pooled_output = self.dropout(pooled_output)

            ## 3개의 레이어를 합치므로 classifier의 차원은 (hidden_dim, 6)이다
            classifier = nn.Linear(pooled_output.shape[1], self.args.num_classes).to(token_ids.device)
            logits = classifier(pooled_output)
        
        else:
            logits=outputs.logits
        
    
        prob= self.softmax(logits)
        # print(prob)
        # logits2 = outputs.logits
        # print(self.softmax(logits2))


        return logits, prob

In [ ]:
# bias model
bias_model = myClassifier(bias_myModel, selected_layers=False, args=args_bias)

In [ ]:
class LossEarlyStopper():
    """Early stopper

        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        # 첫 에폭
        if self.min_loss == np.Inf:
            self.min_loss = loss
           
        # loss가 줄지 않는다면 -> patience_counter 1 증가
        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            # patience 만큼 loss가 줄지 않았다면 학습을 중단합니다.
            if self.patience_counter == self.patience:
                self.stop = True
            print(msg)
        # loss가 줄어듬 -> min_loss 갱신, patience_counter 초기화
        elif loss <= self.min_loss:
            self.patience_counter = 0
            ### v2 에서 수정됨
            ### self.save_model = True -> 삭제 (사용하지 않음)
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss

            print(msg)

In [ ]:
def train(model, train_data, val_data, args, mode = 'train'):
    
    # args.run은 실험 이름 (어디까지나 팀원들간의 버전 관리 및 공유 편의를 위한 것으로, 자유롭게 수정 가능합니다.)
    print("RUN : ", args.run)
    shutil.copyfile(f'{config_path_bias}', os.path.join(args.config_dir, f"config_{args.run}.json"))

    early_stopper = LossEarlyStopper(patience=args.patience)
    
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=args.train_batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=args.train_batch_size)

    print(len(train_dataloader))
    
    if DEBUG == True:
        # 데이터로더가 성공적으로 로드 되었는지 확인
        for idx, data in enumerate(train_dataloader):
            if idx==0:
                print("batch size : ", len(data[0]['input_ids']))
                print("The first batch looks like ..\n", data[0])
    
    
    criterion = nn.CrossEntropyLoss()
    
    total_steps = len(train_dataloader) * args.train_epochs

    optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * args.warmup_proportion), 
                                                num_training_steps=total_steps)

    
    if use_cuda:
        model = model.to(DEVICE)
        criterion = criterion.to(DEVICE)
        

    tr_loss = 0.0
    val_loss = 0.0
    best_score = 0.0
    best_loss =99.0
      

    for epoch_num in range(args.train_epochs):

            total_acc_train = 0
            total_loss_train = 0
            
            assert mode in ['train', 'val'], 'your mode should be either \'train\' or \'val\''
            
            if mode =='train':
                for train_input, train_label in tqdm(train_dataloader):
                    
                    
                    mask = train_input['attention_mask'].to(DEVICE)
                    input_id = train_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = train_input['token_type_ids'].squeeze(1).to(DEVICE)
                    train_label = train_label.long().to(DEVICE)  
                    
                    optimizer.zero_grad()
 
                    output = model(input_id, mask, segment_ids)
                    batch_loss = criterion(output[0].view(-1,3), train_label.view(-1))  # (-1,6) -. (-1,3) 으로 바꿔줌
                    total_loss_train += batch_loss.item()

                    acc = (output[0].argmax(dim=1) == train_label).sum().item()
                    total_acc_train += acc
                    
                    ### v2에 수정됨
                    optimizer.zero_grad()
                    
                    batch_loss.backward()
                    optimizer.step()
                    
                    ### v2 에 수정됨
                    scheduler.step()
                    

            total_acc_val = 0
            total_loss_val = 0
            
            # validation을 위해 이걸 넣으면 이 evaluation 프로세스 중엔 dropout 레이어가 다르가 동작한다.
            model.eval()
            
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    mask = val_input['attention_mask'].to(DEVICE)
                    input_id = val_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = val_input['token_type_ids'].squeeze(1).to(DEVICE)
                    val_label = val_label.long().to(DEVICE)

                    output = model(input_id, mask, segment_ids)
                    batch_loss = criterion(output[0].view(-1,3), val_label.view(-1))
                    total_loss_val += batch_loss.item()
                    
                    acc = (output[0].argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            
            train_loss = total_loss_train / len(train_data)
            train_accuracy = total_acc_train / len(train_data)
            val_loss = total_loss_val / len(val_data)
            val_accuracy = total_acc_val / len(val_data)
            

            # 한 Epoch 학습 후 학습/검증에 대해 loss와 평가지표 (여기서는 accuracy로 임의로 설정) 출력
            print(
                f'Epoch: {epoch_num + 1} \
                | Train Loss: {train_loss: .3f} \
                | Train Accuracy: {train_accuracy: .3f} \
                | Val Loss: {val_loss: .3f} \
                | Val Accuracy: {val_accuracy: .3f}')
          
            # early_stopping check
            early_stopper.check_early_stopping(loss=val_loss)

            if early_stopper.stop:
                print('Early stopped, Best score : ', best_score)
                break

            ### v2 에 수정됨
            ### loss와 accuracy가 꼭 correlate하진 않습니다.
            ### 
            ### 원본 (필요하다면 다시 해제 후 사용)
            # if val_accuracy > best_score : 
            if val_loss < best_loss :
            # 모델이 개선됨 -> 검증 점수와 베스트 loss, weight 갱신
                best_score = val_accuracy 
                best_loss =val_loss
                # 학습된 모델을 저장할 디렉토리 및 모델 이름 지정
                SAVED_MODEL =  os.path.join(args.result_dir, f'best_{args.run}.pt')
            
                check_point = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                }
                torch.save(check_point, SAVED_MODEL)  
              
            # print("scheduler : ", scheduler.state_dict())


    print("train finished")
    return 0

In [ ]:
bias_go = train(bias_model, train_dataset, val_dataset, args_bias, mode = 'train')

In [ ]:
# 테스트 데이터셋 불러오기
test_data = CustomDataset(test_df, tokenizer = BIAS_TOKENIZER, max_len= args_bias.max_seq_len, mode='test')

def bias_test(model, SAVED_MODEL, test_data, args, mode = 'test'):
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=args.eval_batch_size)
    
    if use_cuda:
        model = model.to(DEVICE)
        model.load_state_dict(torch.load(SAVED_MODEL)['model'])

    model.eval()
    pred = []
    with torch.no_grad():
        for test_input in test_dataloader:

            mask = test_input['attention_mask'].to(DEVICE)
            input_id = test_input['input_ids'].squeeze(1).to(DEVICE)
            segment_ids = test_input['token_type_ids'].squeeze(1).to(DEVICE)
            output = model(input_id, mask, segment_ids)
            output = output[0].argmax(dim=1).cpu().tolist()

            for label in output:
                pred.append(label)
    return pred

In [ ]:
test_df.to_csv('/content/drive/MyDrive/AI_team/competition3/data2/hanspell_comment_test.csv', index=False)

In [ ]:
BIAS_SAVED_MODEL =  os.path.join(args_bias.result_dir, f'best_{args_bias.run}.pt')
pred_bias = bias_test(bias_model, BIAS_SAVED_MODEL, test_data, args_bias)

In [ ]:
bias_dict = {0: 'none', 1: 'others', 2: 'gender'}
pred_bias_str = ['' for i in range(len(pred_bias))]
for idx, label in enumerate(pred_bias):
    pred_bias_str[idx]=(str(bias_dict[label]))
print('decode Completed!')

decode Completed!


In [ ]:
# submission 파일에 bias 결과 집어넣기
submit = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/sample_submission.csv')
submit['bias'] = pred_bias_str

In [ ]:
# 제출2번 완전 base 답안
submit_base = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/submission/base_submission.csv')

In [ ]:
'''
base(제출2) 와 BIAS 답안 비교

comment + title 학습 : 37개

comment + title(전처리) : 43개

comment(전처리) : 43개
'''

In [ ]:
len(submit_base[submit_base['bias']!=submit['bias']])

43

## HATE prediction 진행

In [ ]:
# config 파일 불러오기 / hate
config_path_hate = os.path.join(DATA_PATH + '/baseline2/config_hate_stratified.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = AttrDict(json.load(f))
            print("config file loaded.")
            print(args.pretrained_model)
    else:
        assert False, 'config json file cannot be found.. please check the path again.'
    return args
    
# 코드 중간중간에 끼워넣어 리셋 가능
args_hate = set_config(config_path_hate)

# 결과 저장 폴더 미리 생성
os.makedirs(args_hate.result_dir, exist_ok=True)
os.makedirs(args_hate.config_dir, exist_ok=True)

config file loaded.
beomi/beep-KcELECTRA-base-hate


### 전처리 생략해도 되는 부분

In [ ]:
train_path = os.path.join(args_hate.data_dir,'train.csv')
print("train 데이터 경로가 올바른가요? : ", os.path.lexists(train_path))
train_df = pd.read_csv(train_path, encoding = 'UTF-8-SIG')

test_path = os.path.join(args_hate.data_dir,'test.csv')
print("test 데이터 경로가 올바른가요? : ", os.path.lexists(test_path))
test_df = pd.read_csv(test_path)

train 데이터 경로가 올바른가요? :  True
test 데이터 경로가 올바른가요? :  True


In [ ]:
# 학습 df 전처리
train_df['clean_title'] = train_df['title'].apply(lambda x: clean_punc(x, punct, punct_mapping))
# 테스트 df 전처리
test_df['clean_title'] = test_df['title'].apply(lambda x: clean_punc(x, punct, punct_mapping))

### 전처리

In [ ]:
train_df = train_df.drop(['label'], axis=1)

In [ ]:
train_df

,title,comment,bias,hate,clean_comment,clean_comment2
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,김태리 정말 연기잘해 진짜,김태리 정말 연기 잘해 진짜
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,공효진 발연기나이질생각이읍던데 왜계속주연일까,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감...,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,일본 축구 져라 일본 축구 져라 일본 축구 져라,일본 축구 져라 일본 축구 져라 일본 축구 져라
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,난 절대로 임현주 욕하는인간이랑은 안논다,난 절대로 임현주 욕하는 인간이랑은 안 논다
...,...,...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,아니 근데 튜닝한사람은 프러포즈받지도 결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함? ᄏᄏᄏ 자기들은 돈 없어서 ...
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,그러니깐 여자를 잘만나야되 징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯 근데 ...,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none,재미가 없어요 재미가 없어요 재미가 없어요,재미가 없어요 재미가 없어요 재미가 없어요


In [ ]:
# TEST
hate_combinations = np.array(train_df.hate.unique()).T.reshape(-1,1)
hate_labeling = list(np.array([train_df['hate'].values]).T.reshape(-1,1))

hate_labels = []
for i, arr in enumerate(hate_labeling):
    for idx, elem in enumerate(hate_combinations):
        if np.array_equal(elem, arr):
            hate_labels.append(idx)

train_df['label'] = hate_labels

In [ ]:
train_df

,title,comment,bias,hate,clean_comment,clean_comment2,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,김태리 정말 연기잘해 진짜,김태리 정말 연기 잘해 진짜,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,공효진 발연기나이질생각이읍던데 왜계속주연일까,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,누구처럼 돈만 밝히는 저급인생은 살아가지마시길 행복은 머니순이 아니니깐 작은거에 감...,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,1
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,일본 축구 져라 일본 축구 져라 일본 축구 져라,일본 축구 져라 일본 축구 져라 일본 축구 져라,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,난 절대로 임현주 욕하는인간이랑은 안논다,난 절대로 임현주 욕하는 인간이랑은 안 논다,0
...,...,...,...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,아니 근데 튜닝한사람은 프러포즈받지도 결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,아니 근데 튜닝한 사람은 프러포즈 받지도 결혼도 못함? ᄏᄏᄏ 자기들은 돈 없어서 ...,1
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,그러니깐 여자를 잘만나야되 징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯 근데 ...,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...,1
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ♡♡♡,0
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none,재미가 없어요 재미가 없어요 재미가 없어요,재미가 없어요 재미가 없어요 재미가 없어요,0


In [ ]:
HATE_TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer,
    "AutoTokenizer": AutoTokenizer,
    "ElectraTokenizer": ElectraTokenizer,
    "AlbertTokenizer": AlbertTokenizer
}

HATE_TOKENIZER = HATE_TOKENIZER_CLASSES[args_hate.tokenizer_class].from_pretrained(args_hate.pretrained_model)
if DEBUG==True:
    print(HATE_TOKENIZER_CLASSES)

Downloading:   0%|          | 0.00/544 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769k [00:00<?, ?B/s]

{'BertTokenizer': <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>, 'AutoTokenizer': <class 'transformers.models.auto.tokenization_auto.AutoTokenizer'>, 'ElectraTokenizer': <class 'transformers.models.electra.tokenization_electra.ElectraTokenizer'>, 'AlbertTokenizer': <class 'transformers.utils.dummy_sentencepiece_objects.AlbertTokenizer'>}


In [ ]:
# dataset (bias, hate 공통적으로 사용) -> stratified 로 나눠줄 예정
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args_hate.seed)

train_dataset = CustomDataset(train_data, HATE_TOKENIZER, args_hate.max_seq_len, 'train')
val_dataset = CustomDataset(val_data, HATE_TOKENIZER, args_hate.max_seq_len, 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

Train dataset:  7530
Validation dataset:  837


In [ ]:
DEBUG=False

In [ ]:
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification,
    "AutoModel": AutoModel,
    "ElectraForSequenceClassification": ElectraForSequenceClassification,
    "AlbertForSequenceClassification": AlbertForSequenceClassification,

}


hate_myModel = BASE_MODELS[args_bias.architecture].from_pretrained(args_hate.pretrained_model, 
                                                         num_labels = args_hate.num_classes, 
                                                         output_attentions = False, # Whether the model returns attentions weights.
                                                         output_hidden_states = True # Whether the model returns all hidden-states.
                                                        )
if DEBUG==True:
    # 모델 구조 확인
    print(hate_myModel)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

In [ ]:
hate_model = myClassifier(hate_myModel, selected_layers=False, args=args_hate)

In [ ]:
hate_go = train(hate_model, train_dataset, val_dataset, args_hate, mode = 'train')

In [ ]:
# 테스트 데이터셋 불러오기
test_data = CustomDataset(test_df, tokenizer = HATE_TOKENIZER, max_len= args_hate.max_seq_len, mode='test')

def hate_test(model, SAVED_MODEL, test_data, args, mode = 'test'):
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=args.eval_batch_size)
    
    if use_cuda:
        model = model.to(DEVICE)
        model.load_state_dict(torch.load(SAVED_MODEL)['model'])

    model.eval()
    pred = []
    with torch.no_grad():
        for test_input in test_dataloader:

            mask = test_input['attention_mask'].to(DEVICE)
            input_id = test_input['input_ids'].squeeze(1).to(DEVICE)
            segment_ids = test_input['token_type_ids'].squeeze(1).to(DEVICE)
            output = model(input_id, mask, segment_ids)
            output = output[0].argmax(dim=1).cpu().tolist()

            for label in output:
                pred.append(label)
    return pred

In [ ]:
HATE_SAVED_MODEL =  os.path.join(args_hate.result_dir, f'best_{args_hate.run}.pt')
pred_hate = hate_test(hate_model, HATE_SAVED_MODEL, test_data, args_hate)

In [ ]:
hate_dict = {0: 'none', 1: 'hate', 2: 'hate'}   #none offensive hate
pred_hate_str = ['' for i in range(len(pred_hate))]
for idx, label in enumerate(pred_hate):
    pred_hate_str[idx]=(str(hate_dict[label]))
print('decode Completed!')

decode Completed!


In [ ]:
submit['hate'] = pred_hate_str

In [ ]:
'''
base(제출2) 와 hate 답안 비교 기록

comment + title 학습 : 36개

comment + title(전처리) : 28개

comment(전처리) : 26개
'''

In [ ]:
len(submit_base[submit_base['hate']!=submit['hate']])

26

In [ ]:
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,none
507,507,none,none
508,508,others,hate
509,509,none,hate


In [ ]:
submit.to_csv("/content/drive/MyDrive/AI_team/competition3/submission/submission_comment_preproc.csv", index=False)